# Lab 11: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

2022-04-23 19:06:41,592 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

2022-04-23 19:49:30,333 | INFO     |: Simulator is stopped
Loading Flatland...
Initializing pygame framework...
2022-04-23 19:49:31,021 | INFO     |: Plotter is stopped


In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = BaseLocalization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-04-23 19:07:05,764 | INFO     |:  | Number of observations per grid cell: 18
2022-04-23 19:07:05,772 | INFO     |:  | Precaching Views...


/Users/roninsharma/Desktop/Cornell/ECE 5960/Fast-Robots/Lab11/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2022-04-23 19:07:11,160 | INFO     |:  | Precaching Time: 5.386 secs
2022-04-23 19:07:11,161 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-04-23 19:07:11,162 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


In [3]:
# Initialize the Trajectory object
traj = Trajectory(loc)

In [4]:
# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Print the latest observation data stored in the member variable obs_range_data
print(loc.obs_range_data)

[[0.84325863]
 [1.03825932]
 [1.23144788]
 [1.30698932]
 [1.195527  ]
 [1.16208233]
 [1.04429446]
 [0.95093017]
 [2.78488477]
 [1.57397735]
 [3.05917718]
 [2.14224096]
 [1.73569137]
 [1.59070835]
 [0.36424119]
 [0.45904306]
 [0.70412413]
 [0.86945598]]


In [25]:
# In the docstring, "pose" refers to a numpy array with elements (x,y,yaw) in (meters, meters, degrees)

def compute_control(curr_pose, prev_pose):
    """ Given the current and previous odometry poses, this function extracts
    the control information based on the odometry motion model.

    Args:
        cur_pose  ([Pose]): Current Pose
        prev_pose ([Pose]): Previous Pose 

    Returns:
        [delta_rot_1]: Rotation 1  (degrees)
        [delta_trans]: Translation (meters)
        [delta_rot_2]: Rotation 2  (degrees)
    """
    
    delta_x = curr_pose[0] - prev_pose[0]
    delta_y = curr_pose[1] - prev_pose[1]
    delta_thetha = curr_pose[2] - prev_pose[2]
    
    delta_rot_1 = np.degrees( np.arctan2(delta_y, delta_x) - prev_pose[2] )
    delta_rot_2 = np.degrees( delta_thetha - delta_rot_1 )
    
    delta_trans = ( ( delta_x ** 2 ) + ( delta_y ** 2 ) ) ** 0.5

    return mapper.normalize_angle(delta_rot_1), delta_trans, mapper.normalize_angle(delta_rot_2)

def odom_motion_model(cur_pose, prev_pose, u):
    """ Odometry Motion Model

    Args:
        cur_pose  ([Pose]): Current Pose
        prev_pose ([Pose]): Previous Pose
        u (rot1, trans, rot2) (float, float, float): A tuple with control data in the format 
                                                   format (rot1, trans, rot2) with units (degrees, meters, degrees)


    Returns:
        prob [float]: Probability p(x'|x, u)
    """
    
    delta_rot_1, delta_trans, delta_rot_2 = compute_control(cur_pose, prev_pose)
    
    gaussian_rot1 = loc.gaussian(u[0], delta_rot_1, loc.odom_rot_sigma)
    gaussian_rot2 = loc.gaussian(u[2], delta_rot_2, loc.odom_rot_sigma)
    
    gaussian_trans = loc.gaussian(u[1], delta_trans, loc.odom_trans_sigma)
    
    return gaussian_rot1 * gaussian_rot2 * gaussian_trans

def prediction_step(cur_odom, prev_odom):
    """ Prediction step of the Bayes Filter.
    Update the probabilities in loc.bel_bar based on loc.bel from the previous time step and the odometry motion model.

    Args:
        cur_odom  ([Pose]): Current Pose
        prev_odom ([Pose]): Previous Pose
    """
    # previous state: bel_bar
    # current state: bel
    
    maxX = mapper.MAX_CELLS_X
    maxY = mapper.MAX_CELLS_Y
    maxAngle = mapper.MAX_CELLS_A
    
    u = compute_control(cur_odom, prev_odom)
    
    # previous dimensions
    for r in range(maxX):
        for s in range(maxY):
            for t in range(maxAngle):
                
                # skip over small probabilities
                if (loc.bel[r][s][t] > 0.0001):
                
                    prep = mapper.from_map(r, s, t)

                    # current dimensions
                    for c in range(maxX):
                        for d in range(maxY):
                            for e in range(maxAngle):

                                    curp = mapper.from_map(c, d, e)
                                    prob = odom_motion_model(curp, prep, u)
                                    belp = loc.bel[r][s][t]
                                    loc.bel_bar[c][d][e] += prob * belp

    # Special thanks to the lab handout
    loc.bel_bar = loc.bel_bar / np.sum(loc.bel_bar)

def sensor_model(obs):
    """ This is the equivalent of p(z|x).


    Args:
        obs ([ndarray]): A 1D array consisting of the true observations for a specific robot pose in the map 

    Returns:
        float: product of all probabilities (of the likelihoods of each individual sensor measurement)
    """

    prob = 1
    
    for i in range(len(obs)):
        prob *= loc.gaussian(loc.obs_range_data[i][0], obs[i], loc.sensor_sigma)
    
    return prob

def update_step():
    """ Update step of the Bayes Filter.
    Update the probabilities in loc.bel based on loc.bel_bar and the sensor model.
    """
    
    maxX = mapper.MAX_CELLS_X
    maxY = mapper.MAX_CELLS_Y
    maxAngle = mapper.MAX_CELLS_A
    
    # current dimensions
    for c in range(maxX):
        for d in range(maxY):
            for e in range(maxAngle):

                loc.bel[c][d][e] = loc.bel_bar[c][d][e] * sensor_model(mapper.get_views(c, d, e))
    
    # Special thanks to the lab handout
    loc.bel = loc.bel / np.sum(loc.bel)

# Run the Bayes Filter
The cells below utilizes the functions declared above to run each iteration of the Bayes filter algorithm to localize the robot in the grid. <br>

In each iteration of the loop:
- Execute robot motion (get $u_{t}$ as previous and current odom) 
- Perform prediction step (calculate $\overline{bel}$)
- Print information regarding Prediction step
- Execute robot rotation behavior to get observation data (get $z_{t}$)
- Perform update step (calculate $bel$)
- Print information regarding update step


**<ins>NOTE</ins>**: 
- During initial testing, you may want to limit the iteration to only the first time step (i.e t=0) instead of looping through the entire trajectory.
- <span style="color:rgb(0,150,0)">If you make changes to any of the functions above, make sure to re-run the above cells before executing the cell below.</span>
- The functions *print_prediction_stats()* and *print_update_stats()* are helper functions defined in <a href="../localization.py">localization.py</a> and may be changed to suit your needs.
- <span style="color:rgb(0,150,0)">Always run an initial update step before the first prediction step.</span>

#### The cell below contains code to initialize a uniform probability distribution and perform the update step of the Bayes Filter to localize the robot.

In [28]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2022-04-23 19:43:22,203 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-04-23 19:43:22,209 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2022-04-23 19:43:25,653 | INFO     |: ---------- UPDATE STATS -----------
2022-04-23 19:43:25,655 | INFO     |: GT index      : (6, 4, 9)
2022-04-23 19:43:25,657 | INFO     |: Bel index     : (5, 3, 9) with prob = 0.7553703
2022-04-23 19:43:25,659 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-04-23 19:43:25,664 | INFO     |: GT            : (0.000, 0.000, 360.000)
2022-04-23 19:43:25,666 | INFO     |: Belief        : (0.000, -0.305, 10.000)
2022-04-23 19:43:25,669 | INFO     |: POS ERROR     : (-0.000, 0.305, 350.000)
2022-04-23 19:43:25,677 | INFO     |: ---------- UPDATE STATS -----------


In [29]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2022-04-23 19:44:19,636 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-23 19:44:19,649 | INFO     |: GT index         : (6, 3, 7)
2022-04-23 19:44:19,650 | INFO     |: Prior Bel index  : (2, 4, 6) with prob = 0.0575096
2022-04-23 19:44:19,653 | INFO     |: POS ERROR        : (1.196, -0.087, 10.657)
2022-04-23 19:44:19,655 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-23 19:44:22,940 | INFO     |: ---------- UPDATE STATS -----------
2022-04-23 19:44:22,950 | INFO     |: GT index      : (6, 3, 7)
2022-04-23 19:44:22,952 | INFO     |: Bel index     : (6, 4, 6) with prob = 1.0
2022-04-23 19:44:22,959 | INFO     |: Bel_bar prob at index = 3.822943433442566e-05
2022-04-23 19:44:22,969 | INFO     |: GT            : (0.282, -0.087, 320.657)
2022-04-23 19:44:22,970 | INFO     |: Belief        : (0.305, 0.000, -50.000)
2022-04-23 19:44:22,971 | INFO     |: POS ERROR     : (-0.023, -0.087, 370.657)
2022-04-23 19:44:22,973 | IN

2022-04-23 19:45:00,985 | INFO     |: POS ERROR     : (0.160, 0.045, 2526.472)
2022-04-23 19:45:00,988 | INFO     |: ---------- UPDATE STATS -----------
-------------------------------------


----------------- 7 -----------------
2022-04-23 19:45:03,580 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-23 19:45:03,595 | INFO     |: GT index         : (11, 3, 13)
2022-04-23 19:45:03,596 | INFO     |: Prior Bel index  : (10, 4, 1) with prob = 0.0518841
2022-04-23 19:45:03,596 | INFO     |: POS ERROR        : (0.246, -0.209, 2752.108)
2022-04-23 19:45:03,601 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-23 19:45:06,773 | INFO     |: ---------- UPDATE STATS -----------
2022-04-23 19:45:06,779 | INFO     |: GT index      : (11, 3, 13)
2022-04-23 19:45:06,780 | INFO     |: Bel index     : (11, 3, 13) with prob = 1.0
2022-04-23 19:45:06,781 | INFO     |: Bel_bar prob at index = 3.276422391419109e-10
2022-04-23 19:45:06,783 | INFO     |: GT            : (1.770, -0.

2022-04-23 19:45:48,279 | INFO     |: Bel_bar prob at index = 3.7543660476019415e-09
2022-04-23 19:45:48,280 | INFO     |: GT            : (0.036, -0.054, 5268.313)
2022-04-23 19:45:48,283 | INFO     |: Belief        : (0.000, -0.305, -130.000)
2022-04-23 19:45:48,291 | INFO     |: POS ERROR     : (0.036, 0.251, 5398.313)
2022-04-23 19:45:48,294 | INFO     |: ---------- UPDATE STATS -----------
-------------------------------------


----------------- 14 -----------------
2022-04-23 19:45:52,375 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-23 19:45:52,390 | INFO     |: GT index         : (4, 3, 1)
2022-04-23 19:45:52,391 | INFO     |: Prior Bel index  : (3, 3, 14) with prob = 0.0749009
2022-04-23 19:45:52,392 | INFO     |: POS ERROR        : (0.275, 0.118, 5135.295)
2022-04-23 19:45:52,399 | INFO     |: ---------- PREDICTION STATS -----------
2022-04-23 19:45:55,685 | INFO     |: ---------- UPDATE STATS -----------
2022-04-23 19:45:55,690 | INFO     |: GT index      : 